In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.t5 import T5Model

In [3]:
!pip install torch

     |████████████████████████████████| 119.6 MB 7.4 MB/s eta 0:00:01


# Read & Proprocess dataset

In [17]:
dataset = pd.read_csv('Datasets/Emotion_cause_compulation.csv')
dataset = dataset.rename(columns={'text':'input_text','emotion':'target_text'})
dataset.insert(loc=0, column='prefix', value=['multilabel classification']*len(dataset))
train, test = train_test_split(dataset, test_size=0.33, random_state=42)

# Train T5 model

In [8]:
model_args = {
    "max_seq_length": 196,
    "train_batch_size": 16,
    "eval_batch_size": 64,
    "num_train_epochs": 1,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 15000,
    "evaluate_during_training_verbose": True,
    
    "use_multiprocessing": False,
    "fp16": False,

    "save_steps": -1,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,

    "reprocess_input_data": True,
    "overwrite_output_dir": True,

    "wandb_project": "T5 mixed tasks - Binary, Multi-Label, Regression",
}

model = T5Model("t5", "t5-base", args=model_args, use_cuda=False)


In [ ]:
model.train_model(train, eval_data=test)

/Users/yarikmenchaca/opt/anaconda3/envs/transformers/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3255: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  warnings.warn(



Using Adafactor for T5


In [11]:
?model.train_model

Signature:
model.train_model(
    train_data,
    output_dir=None,
    show_running_loss=True,
    args=None,
    eval_data=None,
    verbose=True,
    **kwargs,
)
Docstring:
Trains the model using 'train_data'

Args:
    train_data: Pandas DataFrame containing the 3 columns - `prefix`, `input_text`, `target_text`.
                - `prefix`: A string indicating the task to perform. (E.g. `"question"`, `"stsb"`)
                - `input_text`: The input text sequence. `prefix` is automatically prepended to form the full input. (<prefix>: <input_text>)
                - `target_text`: The target sequence
    output_dir: The directory where model files will be saved. If not given, self.args.output_dir will be used.
    show_running_loss (optional): Set to False to prevent running loss from being printed to console. Defaults to True.
    args (optional): Optional changes to the args dict of the model. Any changes made will persist for the model.
    eval_data (optional): A DataFrame again